In [10]:
import os 


In [11]:
%cd /home/lab/Workspace/e2e-mlflow-dvc

/home/lab/Workspace/e2e-mlflow-dvc


In [12]:
os.environ["MLFLOW_TRACKING_URI"]="https://dagshub.com/diazmanne/e2e-mlflow-dvc.mlflow"
os.environ["MLFLOW_TRACKING_USERNAME"]="diazmanne"
os.environ["MLFLOW_TRACKING_PASSWORD"]="f4e87115c3d98a2ef151c0469443be9738fa35ff"

In [13]:
import tensorflow as tf 

In [14]:
model = tf.keras.models.load_model("artifacts/training/trained_model.keras")

In [15]:
#entity utils/config_entity.py

from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class EvaluationConfig:
    path_of_model: Path
    test_data: Path
    all_params: dict
    mlflow_uri: str
    params_image_size: list
    params_batch_size: int
    

In [16]:
from cnnClassifier.constants import * # Path("config/config.yaml") and Path("params.yaml")
from cnnClassifier.utils.common import read_yaml, create_directories, save_json # methods 

In [20]:
class ConfigurationManager:
    def __init__(
        self, 
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH
    ):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([self.config["artifacts_root"]])
    
    def get_evaluation_config(self) -> EvaluationConfig:
        evaluation_config = self.config["evaluation"]
        return EvaluationConfig(
            path_of_model=Path(evaluation_config["trained_model_path"]),
            test_data=Path(evaluation_config["test_data_path"]),
            all_params=evaluation_config.get("all_params", {}),
            mlflow_uri=self.config["mlflow_uri"],
            params_image_size=self.params["IMAGE_SIZE"],
            params_batch_size=self.params["BATCH_SIZE"]
        )

SyntaxError: 'return' outside function (3186544071.py, line 20)

In [ ]:
import tensorflow as tf
from pathlib import Path
import mlflow
import mlflow.keras
from urllib.parse import urlparse

In [ ]:
class Evaluation:
    def __init__(self, config: EvaluationConfig):
        self.config = config

        
    def _valid_generator(self):

        datagenerator_kwargs = dict(
            rescale = 1./255,
            validation_split=0.30
        )

        dataflow_kwargs = dict(
            target_size=self.config.params_image_size[:-1],
            batch_size=self.config.params_batch_size,
            interpolation="bilinear"
        )

        valid_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
            **datagenerator_kwargs
        )

        self.valid_generator = valid_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset="validation",
            shuffle=False,
            **dataflow_kwargs
        )
    # load and evaluate model     
    @staticmethod
    def load_model(path: Path) -> tf.keras.Model:
        return tf.keras.models.load_model(path)
    

    def evaluation(self):
        self.model = self.load_model(self.config.path_of_model)
        self._valid_generator()
        self.score = model.evaluate(self.valid_generator)
        self.save_score()

    def save_score(self):
        scores = {"loss": self.score[0], "accuracy": self.score[1]}
        save_json(path=Path("scores.json"), data=scores)

    # MLFLOW seup 
    def log_into_mlflow(self):
        mlflow.set_registry_uri(self.config.mlflow_uri)
        tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme
        
        with mlflow.start_run():
            mlflow.log_params(self.config.all_params)
            mlflow.log_metrics(
                {"loss": self.score[0], "accuracy": self.score[1]}
            )
            # Model registry does not work with file store
            if tracking_url_type_store != "file":

                # Register the model
                # There are other ways to use the Model Registry, which depends on the use case,
                # please refer to the doc for more information:
                # https://mlflow.org/docs/latest/model-registry.html#api-workflow
                mlflow.keras.log_model(self.model, "model", registered_model_name="VGG16Model")
            else:
                mlflow.keras.log_model(self.model, "model")

In [ ]:
try:
    config = ConfigurationManager()
    eval_config = config.get_evaluation_config()
    evaluation = Evaluation(eval_config)
    evaluation.evaluation()
    evaluation.log_into_mlflow()

except Exception as e:
   raise e